In [1]:
%load_ext autoreload
%autoreload 2
import warnings
warnings.simplefilter("ignore", UserWarning)


In [2]:
import numpy as np
import torch

In [3]:
train_on_gpu = torch.cuda.is_available()

In [4]:
if not train_on_gpu:
    print('CUDA is not available.  Training on CPU ...')
else:
    print('CUDA is available!  Training on GPU ...')

CUDA is available!  Training on GPU ...


In [5]:
from traing_data import GetSequence

In [6]:
get_train = GetSequence()

In [7]:
import time, math

def time_since(since):
    s = time.time() - since
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

In [8]:
import tqdm

In [13]:
def train(model, model_opt, get_train):
    hidden = model.init_hidden()
#     hidden = hidden.cuda()
    model.zero_grad()
    loss = 0
    for i in tqdm.tqdm_notebook(range(2000)):
        inp, target = get_train.get_train_data()
        inp = inp.to(dtype=torch.float32)
        target = target.to(dtype=torch.float32)
#         inp,target = inp.cuda(), target.cuda()
        target = target.squeeze()
        output, hidden = model(inp, hidden)
        loss += criterion(output, target.unsqueeze(0))

    loss.backward()
    model_opt.step()

    return loss.item() / i

In [14]:
import torch
import torch.nn as nn


In [15]:
from model import RNN

In [ ]:
n_epochs = 10000
print_every = 100
plot_every = 10
hidden_size = 100
n_layers = 1
lr = 0.01

rnn = RNN(n_layers=2)
# rnn.cuda()
print(rnn)
rnn_opt = torch.optim.Adam(rnn.parameters(), lr=lr)
criterion = nn.MSELoss().cuda()

start = time.time()
all_losses = []
loss_avg = 0

for epoch in range(1, n_epochs + 1):
    loss = train(rnn, rnn_opt, get_train)       
    loss_avg += loss

    if epoch % print_every == 0:
        print('[%s (%d %d%%) %.4f]' % (time_since(start), epoch, epoch / n_epochs * 100, loss))
        print(evaluate('Wh', 100), '\n')

    if epoch % plot_every == 0:
        all_losses.append(loss_avg / plot_every)
        loss_avg = 0

RNN(
  (liner): Linear(in_features=10000, out_features=5000, bias=True)
  (gru): GRU(5000, 5000, num_layers=2)
  (decoder): Linear(in_features=5000, out_features=10000, bias=True)
  (softmax): LogSoftmax()
)
